In [1]:
#!conda update -n base -c defaults conda
#!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge geocoder --yes

In [29]:
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import json
#from bs4 import BeautifulSoup
#from sklearn.cluster import KMeans
#import numpy as np
#import re
#import folium
#import os
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#import geocoder

In [11]:
address = 'New York, NY'
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinates of New York, NY are 40.7127281, -74.0060152.


In [12]:
local_filepath = '..\\foursquare_credentials.txt'
f = open(local_filepath, "r")
contents = f.read()
credentials = json.loads(contents)
f.close()

CLIENT_ID = credentials['CLIENT_ID']
CLIENT_SECRET = credentials['CLIENT_SECRET']
VERSION = credentials['VERSION']

In [40]:
LIMIT = 100
def getNearbyVenues(latitudes, longitudes, radius=500):
    
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(
            #lat, 
            #lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [#'Latitude', 
                  #'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [41]:
df = getNearbyVenues([latitude], [longitude])

In [47]:
df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,The Beekman – A Thompson Hotel,40.711173,-74.006702,Hotel
2,Alba Dry Cleaner & Tailor,40.711434,-74.006272,Laundry Service
3,City Hall Park,40.712415,-74.006724,Park
4,Gibney Dance Center Downtown,40.713923,-74.005661,Dance Studio
5,Augustine,40.711310,-74.006660,French Restaurant
6,The Wooly Daily,40.712137,-74.008395,Coffee Shop
7,The Class by Taryn Toomey,40.712753,-74.008734,Gym / Fitness Center
8,CrossFit 212 TriBeCa,40.714537,-74.005999,Gym
9,Takahachi Bakery,40.713653,-74.008804,Bakery


In [51]:
onehot = pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")
fixed_columns = list(onehot.columns)#[onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
print(onehot)

#onehot.head()

    American Restaurant  Antique Shop  Art Gallery  Baby Store  Bakery  Bar  \
0                     0             0            0           0       0    0   
1                     0             0            0           0       0    0   
2                     0             0            0           0       0    0   
3                     0             0            0           0       0    0   
4                     0             0            0           0       0    0   
..                  ...           ...          ...         ...     ...  ...   
95                    0             0            0           0       0    0   
96                    0             0            0           0       0    0   
97                    0             0            0           0       0    0   
98                    0             0            0           0       0    0   
99                    0             0            0           0       0    0   

    Bookstore  Boxing Gym  Bubble Tea Shop  Buildin

In [52]:
onehot.head()

,American Restaurant,Antique Shop,Art Gallery,Baby Store,Bakery,Bar,Bookstore,Boxing Gym,Bubble Tea Shop,Building,...,Speakeasy,Sporting Goods Shop,Strip Club,Sushi Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
